In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
from bs4 import BeautifulSoup # package to transform the data of webpage into pandas dataframe
import requests 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = BeautifulSoup(requests.get(url).content, 'lxml')

In [3]:
table = content.find('table')
td = table.find_all('td')
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighbourhood.append(td[i+2].text.strip())
wiki_df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
wiki_df.columns = ['Postal Code', 'Borough', 'Neighborhood']
wiki_df['Borough'].replace('Not assigned', np.nan, inplace=True)
wiki_df.dropna(subset=['Borough'], inplace=True)
wiki_df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)
wiki_df = wiki_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
wiki_df.columns = ['Postal Code', 'Borough', 'Neighborhood']

In [4]:
wiki_df.shape

(103, 3)

In [5]:
url="http://cocl.us/Geospatial_data"
geospatial_df = pd.read_csv(url)
geospatial_df.columns = ['Postal Code', 'Latitude', 'Longitude']
toronto_metro_df = pd.merge(wiki_df, geospatial_df, on=['Postal Code'], how='inner')
toronto_metro_df .head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_metro_df ['Borough'].unique()),
        toronto_metro_df .shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="coursera-capstone-project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [8]:
toronto_data = toronto_metro_df[toronto_metro_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(
        toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']
        ):
    label = '{}, {}'.format(toronto_data, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto )  
    
map_toronto

In [19]:
CLIENT_ID = '<HR0R1IEP0WDXFYM53HU2GUJLU0GNIEMP4EY5GBSYEN3QGNUW>' 
CLIENT_SECRET = '<GUVXPO0FGF5DNXWWU5GB50GXLFDRAAH5TOZCER1YOVSN0N0F>' 
VERSION = '20180605' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: <HR0R1IEP0WDXFYM53HU2GUJLU0GNIEMP4EY5GBSYEN3QGNUW>
CLIENT_SECRET:<GUVXPO0FGF5DNXWWU5GB50GXLFDRAAH5TOZCER1YOVSN0N0F>


In [20]:
toronto_data.loc[1, 'Neighborhood']

'The Danforth West / Riverdale'

In [31]:
hood_latitude = toronto_data.loc[1, 'Latitude']
hood_longitude = toronto_data.loc[1, 'Longitude']

hood_name = toronto_data.loc[1, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(hood_name, 
                                                               hood_latitude, 
                                                               hood_longitude))

Latitude and longitude values of The Danforth West / Riverdale are 43.6795571, -79.352188.


In [32]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, hood_latitude, hood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5ea7fecb0be7b4001ba35bff'},
 'response': {}}